In [ ]:

%pylab inline

import math
import numpy as np
from scipy.sparse.linalg import inv
#from numpy.linalg import inv
import scipy.sparse as sps
import scipy.sparse.linalg
from scipy import integrate
import sys
import matplotlib.pyplot as plt
sys.path.append('../../src/')
from pylab import *

import parameters as pam
import lattice as lat
import variational_space as vs
import hamiltonian as ham
import basis_change as basis
import get_state as getstate
import utility as util
import plotfig as fig
import ground_state as gs
import lanczos
import time
from tqdm.notebook import tqdm
start_time = time.time()
M_PI = math.pi
                  
#####################################
def compute_Aw_main(A,epx,epy,tpd,tpddiff,tpp,pds,pdsdiff,pdp,pdpdiff,pps,ppp,Upp,Udif,Vpp,Upd,Updf,\
                    d_double,pUppx_double,pUppy_double,pUpp_double,pVpp_double,pUpdx_double,pUpdy_double,U, S_val, Sz_val, AorB_sym): 
    diff = round(abs(epx-epy)*0.5,3)
    if Norb==7:
        fname = 'epx'+str(epx)+'epy'+str(epy)+'_tpd'+str(tpd)+'_tpddiff'+str(tpddiff)+'_tpp'+str(tpp) \
                  +'_A'+str(A)+'_B'+str(pam.B)+'_C'+str(pam.C)+'_Upp'+str(Upp)+'_Udif'+str(Udif)+'_Vpp'\
                  +str(Vpp)+'_Upd'+str(Upd)+'_Updf'+str(Updf) \
                  +'_Mc'+str(Mc)+'_Norb'+str(Norb)+'_eta'+str(eta)
        flowpeak = 'Norb'+str(Norb)+'_tpd'+str(tpd)+'_tpddiff'+str(tpddiff)+'_tpp'+str(tpp)+'_diff'\
                  +str(diff)+'_Upp'+str(Upp)+'_Udif'+str(Udif)\
                  +'_Vpp'+str(Vpp)+'_Upd'+str(Upd)+'_Updf'+str(Updf)+'_Mc'+str(Mc)+'_eta'+str(eta)
    elif Norb==9 or Norb==10 or Norb==11:
        fname = 'epx'+str(epx)+'epy'+str(epy)+'_pds'+str(pds)+'_pdsdiff'+str(pdsdiff)+'_pdp'+str(pdp)\
                  +'_pdpdiff'+str(pdpdiff)+'_pps'+str(pps)+'_ppp'+str(ppp) \
                  +'_A'+str(B)+'_B'+str(pam.B)+'_C'+str(pam.C)+'_Upp'+str(Upp)+'_Udif'+str(Udif)\
                  +'_Vpp'+str(Vpp)+'_Upd'+str(Upd)+'_Updf'+str(Updf)\
                  +'_Mc'+str(Mc)+'_Norb'+str(Norb)+'_eta'+str(eta)
        flowpeak = 'Norb'+str(Norb)+'_pds'+str(pds)+'_pdsdiff'+str(pdsdiff)+'_pdp'+str(pdp)+'_pdpdiff'+str(pdpdiff)\
                  +'_pps'+str(pps)+'_ppp'+str(ppp)+'_diff'+str(diff)+'_Upp'+str(Upp)+'_Udif'+str(Udif)\
                  +'_Vpp'+str(Vpp)+'_Upd'+str(Upd)+'_Updf'+str(Updf)+'_Mc'+str(Mc)+'_eta'+str(eta)
                
    w_vals = np.arange(pam.wmin, pam.wmax, pam.eta/4.0)
   
    Aw = np.zeros(len(w_vals))
    Aw_dd_total = np.zeros(len(w_vals))
    Aw_d8_total = np.zeros(len(w_vals))

    # set up H0
    if Norb==7:
        tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac, tpp_nn_hop_fac \
                                   = ham.set_tpd_tpp(Norb,tpd,tpddiff,tpp,0,0,0,0)
    elif Norb==9 or Norb==11:
        tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac, tpp_nn_hop_fac \
                                   = ham.set_tpd_tpp(Norb,0,0,0,pds,pdsdiff,pdp,pdpdiff,pps,ppp)
            
    T_pd   = ham.create_tpd_nn_matrix(VS,tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac)
    T_pp   = ham.create_tpp_nn_matrix(VS,tpp_nn_hop_fac)  
    Esite  = ham.create_edep_diag_matrix(VS,A,epx,epy)
    
    H0 = T_pd + T_pp + Esite  
    
    '''
    Below probably not necessary to do the rotation by multiplying U and U_d
    the basis_change.py is only for label the state as singlet or triplet
    and assign the interaction matrix
    '''
    if pam.if_H0_rotate_byU==1:
        H0_new = U_d.dot(H0.dot(U))
    
    clf()

    if Norb==7 or Norb==9 or Norb==10 or Norb==11:     
        Hint = ham.create_interaction_matrix_ALL_syms(VS,d_double,pUppx_double,pUppy_double,pUpp_double,pVpp_double,pUpdx_double,\
                                                      pUpdy_double,S_val, Sz_val,AorB_sym,A,Upp,Udif,Vpp,Upd,Updf)
        if pam.if_H0_rotate_byU==1:
            H = H0_new + Hint 
        else:
            H = H0 + Hint 
        H.tocsr()

        ####################################################################################
        # compute GS only for turning on full interactions
        if pam.if_get_ground_state==1:
            vals, vecs, wgt_d8, wgt_d9L, wgt_d10L2 = gs.get_ground_state(H, VS, S_val, Sz_val)
            if Norb==7:
#                 util.write_GS('Egs_'+flowpeak+'.txt',A,epx,epy,tpd,vals[0])
                util.write_GS_components('GS_weights_'+flowpeak+'.txt',A,epx,epy,tpd,wgt_d8, wgt_d9L, wgt_d10L2)
            elif Norb==9 or Norb==10 or Norb==11:
#                 util.write_GS2('Egs_'+flowpeak+'.txt',A,epx,epy,pds,pdp,vals[0])
                util.write_GS_components2('GS_weights_'+flowpeak+'.txt',A,epx,epy,pds,pdp,wgt_d8, wgt_d9L, wgt_d10L2)
            
        #########################################################################
        '''
        Compute A(w) for various states
        '''
        # compute d8
#         fig.compute_Aw_d8_sym(H, VS, d_double, S_val, Sz_val, AorB_sym, A, w_vals, "Aw_d8_sym_", fname)
        
        # compute d9L
#         b1L_state_indices, a1L_state_indices, b1L_state_labels, a1L_state_labels \
#                 = getstate.get_d9L_state_indices(VS, S_val, Sz_val)
#         fig.compute_Aw1(H, VS, w_vals, b1L_state_indices, b1L_state_labels, "Aw_b1L_", fname)
#         fig.compute_Aw1(H, VS, w_vals, a1L_state_indices, a1L_state_labels, "Aw_a1L_", fname)
        
#         # compute d10L2
#         d10L2_state_indices, d10L2_state_labels = getstate.get_d10L2_state_indices(VS, S_val, Sz_val)
#         fig.compute_Aw1(H, VS, w_vals, d10L2_state_indices, d10L2_state_labels, "Aw_d10L2_", fname)
        
##########################################################################
if __name__ == '__main__': 
    Mc  = pam.Mc
    print ('Mc=',Mc)

    Norb = pam.Norb
    eta  = pam.eta
    ed   = pam.ed

    As = pam.As
    B  = pam.B
    C  = pam.C
    
    # set up VS
    VS = vs.VariationalSpace(Mc)
    basis.count_VS(VS)
    
    d_double,pUppx_double,pUppy_double,pUpp_double, pVpp_double, pUpdx_double,pUpdy_double = ham.get_double_occu_list(VS)
    
    # change the basis for d_double states to be singlet/triplet
    if pam.basis_change_type=='all_states':
        U, S_val, Sz_val, AorB_sym = basis.create_singlet_triplet_basis_change_matrix(VS,d_double)
       
        if pam.if_print_VS_after_basis_change==1:
            basis.print_VS_after_basis_change(VS,S_val,Sz_val)
    elif pam.basis_change_type=='d_double':
        U, S_val, Sz_val, AorB_sym = basis.create_singlet_triplet_basis_change_matrix_d_double(VS,d_double)

    U_d = (U.conjugate()).transpose()
    # check if U if unitary
    #checkU_unitary(U,U_d)
    
    if Norb==7:
        
        for tpd in tqdm(pam.tpds):
            for tpddiff in pam.tpddiffs:
                tpddiff = round(tpddiff,3)
                for ep1 in pam.ep_avgs: 
                    for ep2 in pam.ep_difs: 
                        epx = ep1+ep2
                        epy = ep1-ep2
                        for A in pam.As:
                            util.get_atomic_d8_energy(A,B,C)
                            for tpp in pam.tpps:
                                for Upp in pam.Upps:
                                    Upp = round(Upp,3)
                                    for Udif in pam.Udifs:
                                        Udif = round(Udif,3)
                                        for Vpp in pam.Vpps:
                                            Vpp = round(Vpp,3)
                                            for Upd in pam.Upds:
                                                Upd = round(Upd,3)
                                                for Updf in pam.Updfs:
                                                    Updf = round(Updf,3) 
                                                    print ('===================================================')
                                                    print ('A=',A, 'epx=', epx, 'epy=', epy, ' tpd=',tpd,' tpddiff=',tpddiff,' tpp=',tpp, \
                                                           ' Upp=',Upp,'Udif=',Udif,'Vpp=',Vpp,'Upd=',Upd,'Updf=',Updf)
                                                    compute_Aw_main(A,epx,epy,tpd,tpddiff,tpp,0,0,0,0,0,0,Upp,Udif,Vpp,Upd,Updf,\
                                                                    d_double,pUppx_double,pUppy_double,pUpp_double,\
                                                                    pVpp_double,pUpdx_double,\
                                                                    pUpdy_double,U, S_val, Sz_val, AorB_sym)

                                
    elif Norb==9 or Norb==10 or Norb==11:
        total_iterations =len(pam.pdps)*len(pam.pdsdiffs)*len(pam.ep_avgs)*len(pam.ep_difs)*len(pam.As)*\
        len(pam.Upps)*len(pam.Udifs)*len(pam.Vpps)*len(pam.Upds)*len(pam.Updfs)
        pps = pam.pps
        ppp = pam.ppp
        with tqdm(total=total_iterations, desc="Total Progress") as pbar:
            for ii in range(0,len(pam.pdps)):
                pds = pam.pdss[ii]
                pds = round(pds,5)
                pdp = pam.pdps[ii]
                for pdsdiff in pam.pdsdiffs:
                    pdpdiff = round(pdsdiff*np.sqrt(3)/4,5)
                    pdsdiff = round(pdsdiff,3)
                    for ep1 in pam.ep_avgs: 
                        for ep2 in pam.ep_difs: 
                            epx = ep1+ep2
                            epy = ep1-ep2
                            for A in pam.As:
                                util.get_atomic_d8_energy(A,B,C)
                                for Upp in pam.Upps:
                                    Upp = round(Upp,3)
                                    for Udif in pam.Udifs:
                                        Udif = round(Udif,3)
                                        for Vpp in pam.Vpps:
                                            Vpp = round(Vpp,3)
                                            for Upd in pam.Upds:
                                                Upd = round(Upd,3)
                                                for Updf in pam.Updfs:
                                                    Updf = round(Updf,3)
                                                    print ('===================================================')
                                                    print ('A=',A, 'epx=', epx, 'epy=', epy,' pds=',pds,' pdp=',pdp,\
                                                           ' pps=',pps,' ppp=',ppp, ' Upp=',Upp, ' Udif=',Udif,' Vpp=',Vpp ,' Upd=',Upd,)
                                                    compute_Aw_main(A,epx,epy,0,0,0,pds,pdsdiff,pdp,pdpdiff,pps,ppp,Upp,Udif,Vpp,Upd,Updf,\
                                                                    d_double,pUppx_double,pUppy_double,pUpp_double,\
                                                                    pVpp_double,pUpdx_double,\
                                                                    pUpdy_double,\
                                                                    U, S_val, Sz_val, AorB_sym)  
                                                    pbar.update(1)


    print("--- %s seconds ---" % (time.time() - start_time))

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
Ni_orbs =  ['d3z2r2', 'dx2y2', 'dxy', 'dxz', 'dyz']
O1_orbs =  ['px1', 'py1']
O2_orbs =  ['px2', 'py2']
compute A(w) for symmetries =  ['1A1', '1B1', '3B1', '1A2', '3A2', '1E', '3E']
Mc= 10
VS.dim =  42025
No. of states with count_upup, count_updn, count_dnup, count_dndn: 0 21225 20800 0
len(d_list) 25
len(pUpp_list) 0
len(pUppx_list) 200
len(pUppy_list) 200
len(pVpp_list) 1440
len(pUpdx_list) 40
len(pUpdy_list) 40


Total Progress:   0%|          | 0/495 [00:00<?, ?it/s]

E_1S =  12.16
E_1G =  7.76
E_1D =  6.71
E_3P =  7.05
E_3F =  4.8
A= 6.0 epx= 0.0 epy= 0.0  pds= 1.50111  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0.0  Udif= 0.0  Vpp= 0.0  Upd= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-6.36779908 -5.51111857 -5.51111857 -5.4638228  -5.4638228  -5.4638228
 -5.45641679 -5.45055066 -5.38926568 -5.38926568]
eigenvalue =  -6.367799078405787
Compute the weights in GS (lowest Aw peak)
no e-h state  6650 px1 -2 -1 0 py2 -1 0 0 S= 0 Sz= 0 , weight =  0.0025987066962827616
no e-h state  6658 px1 -2 1 0 py2 -1 0 0 S= 0 Sz= 0 , weight =  0.002598706696282769
no e-h state  6712 px1 -1 -2 0 px1 -1 0 0 S= 0 Sz= 0 , weight =  0.002948391717735362
no e-h state  6720 px1 -1 0 0 px1 -1 0 0 S= 0 Sz= 0 , weight =  0.02065497043985913
no e-h state  7044 px1 -1 0 0 px1 -1 2 0 S= 0 Sz= 0 , weight =  0.0029483917177353662
no e-h state  9858 py2 -2 -1 0 py2 0 -1 

sumweight= 0.9060573763720562
E_1S =  12.16
E_1G =  7.76
E_1D =  6.71
E_3P =  7.05
E_3F =  4.8
A= 6.0 epx= 1.0 epy= 1.0  pds= 1.50111  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0.0  Udif= 0.0  Vpp= 0.0  Upd= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-4.88965404 -3.90543317 -3.90543317 -3.85813739 -3.85813739 -3.85813739
 -3.85287406 -3.84579986 -3.78358027 -3.78358027]
eigenvalue =  -4.889654039822192
Compute the weights in GS (lowest Aw peak)
no e-h state  6720 px1 -1 0 0 px1 -1 0 0 S= 0 Sz= 0 , weight =  0.017991927115688148
no e-h state  9920 px1 -1 -2 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.002645870833350246
no e-h state  9928 px1 -1 0 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.04370322928106017
no e-h state  9936 px1 -1 2 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.0026458708333502467
no e-h state  9999 py2 0 -1 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.026047635559124725
no e-h stat

sumweight= 0.8926152659382621
E_1S =  12.16
E_1G =  7.76
E_1D =  6.71
E_3P =  7.05
E_3F =  4.8
A= 6.0 epx= 2.5 epy= 2.5  pds= 1.50111  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0.0  Udif= 0.0  Vpp= 0.0  Upd= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-2.86228249 -1.69099587 -1.69099587 -1.6437001  -1.6437001  -1.6437001
 -1.64157712 -1.63262838 -1.56914298 -1.56914298]
eigenvalue =  -2.8622824895544174
Compute the weights in GS (lowest Aw peak)
no e-h state  6720 px1 -1 0 0 px1 -1 0 0 S= 0 Sz= 0 , weight =  0.013583715837346444
no e-h state  9928 px1 -1 0 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.03401581799039158
no e-h state  9999 py2 0 -1 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.020632321552087794
no e-h state  10626 dx2y2 -2 -1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.006596471874860257
no e-h state  10646 dx2y2 -2 1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.006596471874860257
no e-h s

...

sumweight= 0.9873921689543452
E_1S =  12.16
E_1G =  7.76
E_1D =  6.71
E_3P =  7.05
E_3F =  4.8
A= 6.0 epx= 8.399999999999993 epy= 8.399999999999993  pds= 1.50111  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0.0  Udif= 0.0  Vpp= 0.0  Upd= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[2.73798124 3.03714002 3.22352496 3.25309778 3.62470972 4.01711525
 4.0245559  4.42014336 4.42199557 4.42335351]
eigenvalue =  2.7379812400268855
Compute the weights in GS (lowest Aw peak)
no e-h state  10801 d3z2r2 -1 0 0 px1 0 0 0 S= 1 Sz= 0 , weight =  0.06847701905094537
no e-h state  10805 dx2y2 -1 0 0 px1 0 0 0 S= 1 Sz= 0 , weight =  0.01325475817530439
no e-h state  10983 d3z2r2 0 -1 0 py2 0 0 0 S= 1 Sz= 0 , weight =  0.06440037265539865
no e-h state  10987 dx2y2 0 -1 0 py2 0 0 0 S= 1 Sz= 0 , weight =  0.011956845961225706
no e-h state  11005 d3z2r2 0 0 0 dx2y2 0 0 0 S= 1 Sz= 0 , weight =  0.66

sumweight= 0.9830764612212275
E_1S =  12.16
E_1G =  7.76
E_1D =  6.71
E_3P =  7.05
E_3F =  4.8
A= 6.0 epx= 9.0 epy= 9.0  pds= 1.50111  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0.0  Udif= 0.0  Vpp= 0.0  Upd= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[2.93225977 3.39043765 3.42036186 3.44303876 3.8330646  4.09727305
 4.33624019 4.46284451 4.46412425 4.77259249]
eigenvalue =  2.932259771141383
Compute the weights in GS (lowest Aw peak)
no e-h state  10801 d3z2r2 -1 0 0 px1 0 0 0 S= 1 Sz= 0 , weight =  0.05980335223430122
no e-h state  10805 dx2y2 -1 0 0 px1 0 0 0 S= 1 Sz= 0 , weight =  0.011863113797996707
no e-h state  10983 d3z2r2 0 -1 0 py2 0 0 0 S= 1 Sz= 0 , weight =  0.05605633543991144
no e-h state  10987 dx2y2 0 -1 0 py2 0 0 0 S= 1 Sz= 0 , weight =  0.010653321080962633
no e-h state  11005 d3z2r2 0 0 0 dx2y2 0 0 0 S= 1 Sz= 0 , weight =  0.7071338836036897
no e-h state 

sumweight= 0.9949010852660501
E_1S =  12.16
E_1G =  7.76
E_1D =  6.71
E_3P =  7.05
E_3F =  4.8
A= 6.0 epx= 12.0 epy= 12.0  pds= 1.50111  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0.0  Udif= 0.0  Vpp= 0.0  Upd= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[3.57576781 3.91729571 3.94441727 4.32689119 4.38287611 4.58163903
 4.58259354 4.9001476  5.3764694  5.72181782]
eigenvalue =  3.5757678129854997
Compute the weights in GS (lowest Aw peak)
no e-h state  10801 d3z2r2 -1 0 0 px1 0 0 0 S= 1 Sz= 0 , weight =  0.030433832080551785
no e-h state  10805 dx2y2 -1 0 0 px1 0 0 0 S= 1 Sz= 0 , weight =  0.006772071349968551
no e-h state  10983 d3z2r2 0 -1 0 py2 0 0 0 S= 1 Sz= 0 , weight =  0.028177232106048354
no e-h state  10987 dx2y2 0 -1 0 py2 0 0 0 S= 1 Sz= 0 , weight =  0.0060251388027428125
no e-h state  11005 d3z2r2 0 0 0 dx2y2 0 0 0 S= 1 Sz= 0 , weight =  0.8530474758027781
no e-h 

sumweight= 0.9057282518562202
E_1S =  12.16
E_1G =  7.76
E_1D =  6.71
E_3P =  7.05
E_3F =  4.8
A= 6.0 epx= 1.0 epy= 1.0  pds= 1.50111  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0.0  Udif= 0.0  Vpp= 0.0  Upd= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-4.88231524 -3.90034726 -3.90034726 -3.85305149 -3.85305149 -3.85305149
 -3.84773641 -3.84068388 -3.77849437 -3.77849437]
eigenvalue =  -4.882315242356848
Compute the weights in GS (lowest Aw peak)
no e-h state  6720 px1 -1 0 0 px1 -1 0 0 S= 0 Sz= 0 , weight =  0.024340825807717553
no e-h state  9928 px1 -1 0 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.043951219281298394
no e-h state  9999 py2 0 -1 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.019491317224803476
no e-h state  10626 dx2y2 -2 -1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.007266007006786075
no e-h state  10646 dx2y2 -2 1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.007266007006786068
no e-h 

start create_tpp_nn_matrix
